In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import re

# Final project - Nate Cook

The main question of this notebook is can I predict the amount of arrests per game based on various factors. Some other questions I have is what team has the most average arrests per game, what factors lead to the most arrests, and what day has the most arrests on average.

## 1 - Looking at the data

### 1.1 - Data Description
- `season`: The NFL season year. Int
- `week_num`: The week number of the NFL season. Int
- `day_of_week`: The day of the week the game was played. String
- `gametime_local`: The local time the game started. String
- `home_team`: The name of the home team. String
- `away_team`: The name of the away team. String
- `home_score`: The score of the home team. Int
- `away_score`: The score of the away team. Int
- `OT_flag`: Indicates if the game went into overtime. String (e.g., "OT" or empty)
- `arrests`: The number of arrests made during and after the game. Int
- `division_game`: Indicates if the game was a divisional matchup. String (e.g., "y" or "n")

### 1.2 - Data overview

In [ ]:
# Load data
df = pd.read_csv("nfl_arrests_2011-2015.csv")
print(f"Dataset shape: {df.shape}")

In [ ]:
# Display the first 10 rows
display(df.head(10))

In [ ]:
# Describe the data
df.describe()

In [ ]:
# Get the info of the data
df.info()

From this overview of the data my first big problem appears with the amount of categorical features. There is a heavy amount of categorical data I need to convert, but most of it shouldn't be bad. I could convert the ot flag to 0 if false and 1 if true and do the same for division game. I can convert game time and day to integers as well, but converting teams through one hot encoding might make the data set to large. I need to research some ideas.

### 1.5 - Check for nulls

In [ ]:
# Check for null values
nulls = df.isnull().sum()
print("\nNull values per column:")
print(nulls)

A lot of nulls in OT_Flag but that will be fixed in cleaning and arrests will just be filled with mean in cleaning as well

### 1.7 - Finding basic stats

In [ ]:
# Games per season
print("\nGames per season:")
display(df['season'].value_counts().sort_index())

In [ ]:
# Games per day
print("\nGames per day of week:")
display(df['day_of_week'].value_counts())

In [ ]:
# Mean arrests per game
print("\nMean arrests per game:", df['arrests'].mean())

## 2 - Cleaning the data

This is a hard section for me because I want to fill nulls, convert, add features, and scale them all before I can start to run predications on anything

### 2.1 - Fill null values

In [ ]:
# Fill null ot flag values with N
df['OT_flag'] = df['OT_flag'].fillna('N')

# Check to see if it worked
nulls = df.isnull().sum()
print("\nNull values per column:")
print(nulls)

In [ ]:
# Next fill all numerical features with mean
numeric_columns = df.select_dtypes(include=[np.number]).columns
imputer = SimpleImputer(strategy='mean')
df[numeric_columns] = imputer.fit_transform(df[numeric_columns])

# Check to see if it worked
nulls = df.isnull().sum()
print("\nNull values per column:")
print(nulls)

Now all nulls are filled, so now we can start to convert

### 2.5 - Convert categorical to numerical

First im going to start with one hot encoding the day of the week

In [ ]:
df["day_of_week"].value_counts()

5 Values means 5 features will be add, but it is very annoying that I have to add wednesday as a feature even though it only appears once

In [ ]:
# Day of week to numerical
day_encoder = OneHotEncoder(sparse_output=False)
day_encoded = day_encoder.fit_transform(df[['day_of_week']])
day_encoded_df = pd.DataFrame(day_encoded, columns=[f'day_{cat}' for cat in day_encoder.categories_[0]])
df = pd.concat([df, day_encoded_df], axis=1)
df.drop('day_of_week', axis=1, inplace=True)

display(df.head(10))

Next is teams

In [ ]:
df["home_team"].value_counts()

There is defiantly too many teams here to one hot encode because I have to do away and home. After researching only I think label encoding will work the best out of my options. How this will work is it will assign each home and away team a integer value that will correspond to the team. It shouldn't be to bad to make it work for this.

In [ ]:
# Team encoding
all_teams = sorted(set(df['home_team'].unique()).union(set(df['away_team'].unique())))
team_to_id = {team: idx for idx, team in enumerate(all_teams)}

df['home_team_id'] = df['home_team'].map(team_to_id)
df['away_team_id'] = df['away_team'].map(team_to_id)
df.drop(['home_team', 'away_team'], axis=1, inplace=True)

display(df.head(10))

In [ ]:
# Print out the table of the id with the team
team_encoding_df = pd.DataFrame(list(team_to_id.items()), columns=['Team', 'Team_ID'])
display(team_encoding_df.sort_values('Team_ID'))

Label encoding worked great here and now I just have to fix a couple more values and I should be done with converting soon

In [ ]:
# Convert OT_flag and division_game to 0 and 1
df['overtime'] = df['OT_flag'].apply(lambda x: 1 if x == 'OT' else 0)
df.drop('OT_flag', axis=1, inplace=True)

# Convert division_game
df['division_game'] = df['division_game'].apply(lambda x: 1 if x.lower() == 'y' else 0)

# Check if worked
display(df.head(10))

Lastly I need to convert game time to numerical. This turned out to be pretty hard because I tried to convert to hours and just round, but that didn't work at all and messed up the data anyways. After looking online for a bit I found the re library and a stack overflow question with basically the same problem and just used the function in there and it worked great.

In [ ]:
# Convert game time
def time_to_minutes(time_str):
    match = re.search(r'(\d+):(\d+):(\d+)\s*([AP]M)', time_str)
    if match:
        hour, minute, second, period = match.groups()
        hour = int(hour)
        if period == 'PM' and hour < 12:
            hour += 12
        elif period == 'AM' and hour == 12:
            hour = 0
        return hour * 60 + int(minute)
    return None

df['game_minutes'] = df['gametime_local'].apply(time_to_minutes)
df.drop('gametime_local', axis=1, inplace=True)

# Check if worked
display(df.head(10))

Now after all the conversion I got every feature to numerical and I didn't have to use one hot encoding much so the data set isn't extremely big either. Now I want to add a couple more features to help the future analysis.

### 2.7 - Feature engineering

For the features I decided I want to add I went with total points, score difference, home team win, high scoring, and game competitiveness all of which will be explained in more code comments when in their cells.

In [ ]:
# First is total points which is just home + away points added together
df['total_points'] = df['home_score'] + df['away_score']

In [ ]:
# Next is score difference which is the difference between the two teams scoring
df['score_diff'] = df['home_score'] - df['away_score']

In [ ]:
# Now home team win is added which is just seeing whether the home team won and if they did a 1 is recorded and if not a 0 is recorded
df['home_win'] = (df['score_diff'] > 0).astype(int)

In [ ]:
# Next is high scoring which finds the average score of the games and if it is above records a 1 and if not it records a 0
median_score = df['total_points'].median()
df['high_scoring'] = (df['total_points'] > median_score).astype(int)

In [ ]:
# Lastly is game competitiveness which uses how close the score was to find if they game was competitive
df['competitive_game'] = 1 - (abs(df['score_diff']) / df['total_points'])

Now we can check if they all worked

In [ ]:
display(df.head(10))

It looks perfect and now that those features are added its finally time to scale

### 2.7 - Splitting and Scaling

I forgot I need to split my data before so im going to do that quickly and then scale

In [ ]:
# Define target variable and features
y = df['arrests']
X = df.drop('arrests', axis=1)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")

Now I can actually scale

In [ ]:
# Define which ones to scale because no point in scale the 0 and 1 columns
cols_to_scale = ['season', 'week_num', 'home_score', 'away_score', 
                'game_minutes', 'total_points', 'score_diff', 'competitive_game']

# Create and fit scaler using training data
scaler = StandardScaler()
X_train[cols_to_scale] = scaler.fit_transform(X_train[cols_to_scale])

# Apply the same to the test data
X_test[cols_to_scale] = scaler.transform(X_test[cols_to_scale])


Now that the training and test data is scaled I can output the head for each and see if I can finally be done with cleaning

In [ ]:
print("Final training data after cleaning: ")
display(X_train.head(3))

print("Final test data after cleaning: ")
display(X_test.head(3))

It looks good and now I am finally done with cleaning.

A quick summary of what I did with part 2 was I filled all nulls, converted all categorical data to numerical with one hot encoding and label encoding, I split the data into test and training data and made sure my target (arrests) was not included, I also added 5 new features to help with future analysis, and lastly I scaled both the test and training data. 

## 3 - Predictions

### 3.1 - Feature correlation and Distributions

Firstly, I want to look at the target variable more closely to help my future predictions

In [ ]:
# Visualize arrests
import matplotlib.pyplot as plt
plt.hist(y_train, bins=20, edgecolor='black')
plt.title('Distribution of Arrests')
plt.xlabel('Number of Arrests')
plt.ylabel('Frequency')
plt.show()

From the graph you can see the mass majority of the arrests fall into the 0 - 10 range. This will be important in the future

Next, I want to make a feature correlation to see which features impact it the most

In [ ]:
# Make the correlation matrix and the features
correlation_matrix = pd.concat([X_train, y_train], axis=1).corr()
top_features = correlation_matrix['arrests'].abs().sort_values(ascending=False)[1:11].index.tolist()
top_corr = correlation_matrix.loc[top_features + ['arrests'], top_features + ['arrests']]

# Plot it
plt.figure(figsize=(10, 8))
sns.heatmap(top_corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap: Top 10 Features vs Arrests')
plt.show()

From this graph you can see a lot of interesting features. Home team winning, total points, game minutes, and away score lead to a lot of violence the more they increase. While oddly Sunday seems to see a lot less arrests than Monday games for example.

In [ ]:
correlation_matrix = X_train.corr()
target_correlations = pd.concat([X_train, y_train], axis=1).corr()['arrests'].sort_values(key=abs, ascending=False)
print("Features most correlated with arrests:")
print(target_correlations[1:11])

Now after learning more about the data and features im ready to start with training the model

### 3.5 - Linear regression first attempts

Im going to start with linear regression and depending on how it does I wil try and improve or change to a different model

In [ ]:
# Simple linear regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)

print("Linear Regression Results:")
print(f"RMSE: {root_mean_squared_error(y_test, lr_pred):.3f}")
print(f"MAE: {mean_absolute_error(y_test, lr_pred):.3f}")
print(f"R2: {r2_score(y_test, lr_pred):.3f}")

In [ ]:
plt.figure(figsize=(7, 7))
sns.scatterplot(x=y_test, y=lr_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect Prediction')
plt.xlabel('Actual Arrests')
plt.ylabel('Predicted Arrests')
plt.title('Actual vs Predicted Arrests (Linear Regression)')
plt.legend()
plt.tight_layout()
plt.show()

Linear regression definitely doesn't work. Im going to try random forest to see if that can help

In [ ]:
# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

print("Random Forest Results:")
print(f"RMSE: {root_mean_squared_error(y_test, rf_pred):.3f}")
print(f"MAE: {mean_absolute_error(y_test, rf_pred):.3f}")
print(f"R²: {r2_score(y_test, rf_pred):.3f}")

In [ ]:
plt.figure(figsize=(7, 7))
sns.scatterplot(x=y_test, y=rf_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect Prediction')
plt.xlabel('Actual Arrests')
plt.ylabel('Predicted Arrests')
plt.title('Actual vs Predicted Arrests (Random forest)')
plt.legend()
plt.tight_layout()
plt.show()

Trying to use regression on this data clearly won't work, so instead im going to try to split the arrests into low arrests games and high arrests games and then predict on these separate category's.

### 3.7 - More attempts

So to make two separate category's I need to create labels of what a high arrest game is, and I will be going with above the 75th percentile. Then I just split the games into the high vs normal depending on where they fall in the quantile range.

In [ ]:
# Define the threshold and split the training and test sets
high_arrest_threshold = y_train.quantile(0.75)
y_train_class = (y_train > high_arrest_threshold).astype(int)
y_test_class = (y_test > high_arrest_threshold).astype(int)

In [ ]:
# Split the test games into their groups
normal_games = X_test[y_test_class == 0].copy()
high_arrest_games = X_test[y_test_class == 1].copy()

# Add arrests for reference
normal_games['arrests'] = y_test[y_test_class == 0]
high_arrest_games['arrests'] = y_test[y_test_class == 1]

Perfect, now im going to predict on normal and high separately.

In [ ]:
# Use split based on the threshold
normal_test_idx = y_test_class == 0
high_test_idx = y_test_class == 1

# Make a prediction on both normal and high games
arrest_pred = np.zeros_like(y_test, dtype=float)
arrest_pred[normal_test_idx] = reg_normal.predict(X_test[normal_test_idx])
arrest_pred[high_test_idx] = reg_high.predict(X_test[high_test_idx])

In [ ]:
# Class labels for test set
normal_mask = y_test_class == 0
high_mask = y_test_class == 1

print("Normal Games Regression Results:")
print(f"  RMSE: {root_mean_squared_error(y_test[normal_mask], arrest_pred[normal_mask]):.3f}")
print(f"  MAE: {mean_absolute_error(y_test[normal_mask], arrest_pred[normal_mask]):.3f}")
print(f"  R2: {r2_score(y_test[normal_mask], arrest_pred[normal_mask]):.3f}")

print("\nHigh-Arrest Games Regression Results:")
print(f"  RMSE: {root_mean_squared_error(y_test[high_mask], arrest_pred[high_mask]):.3f}")
print(f"  MAE: {mean_absolute_error(y_test[high_mask], arrest_pred[high_mask]):.3f}")
print(f"  R2: {r2_score(y_test[high_mask], arrest_pred[high_mask]):.3f}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Normal games
axes[0].scatter(y_test[normal_mask], arrest_pred[normal_mask], alpha=0.7, color='blue')
axes[0].plot([y_test[normal_mask].min(), y_test[normal_mask].max()],
             [y_test[normal_mask].min(), y_test[normal_mask].max()],
             'r--', label='Perfect Prediction')
axes[0].set_title('Normal Games')
axes[0].set_xlabel('Actual Arrests')
axes[0].set_ylabel('Predicted Arrests')
axes[0].legend()

# High-Arrest games
axes[1].scatter(y_test[high_mask], arrest_pred[high_mask], alpha=0.7, color='orange')
axes[1].plot([y_test[high_mask].min(), y_test[high_mask].max()],
             [y_test[high_mask].min(), y_test[high_mask].max()],
             'r--', label='Perfect Prediction')
axes[1].set_title('High-Arrest Games')
axes[1].set_xlabel('Actual Arrests')
axes[1].legend()

plt.suptitle('Actual vs Predicted Arrests by Game Category')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

The problem still exists of outliers skewing the prediction to much and ive accepted that arrests is not really predictable with the data in the dataset. No feature on amount of attendance, No weather data (Worse weather likely leads to less arrests), if the game is a playoff game, etc. Instead I will switch from trying to predict amount of arrests to a classification of if a game will be high arrest or a normal amount of arrests. This will be similar to how I split the data because im defining a high arrest game above the 75th percentile, but I will then need to actually predict. Also I will be using random forest's classifier for the predictions.

### 3.9 - Classifier 

In [ ]:
# Define the high arrest games (75th percentile)
high_arrest_threshold = y_train.quantile(0.75)
print(f"High-arrest threshold: {high_arrest_threshold}")

# Create binary target
y_train_class = (y_train > high_arrest_threshold).astype(int)
y_test_class = (y_test > high_arrest_threshold).astype(int)

In [ ]:
# Make prediction using random forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train_class)
y_pred_class = clf.predict(X_test)

# Print metrics
print("Classification Report (Normal vs High-Arrest):")
print(classification_report(y_test_class, y_pred_class, target_names=["Normal", "High-Arrest"]))
print("Confusion Matrix:")
print(confusion_matrix(y_test_class, y_pred_class))

Im kind of happy with the classification report as it shows scores in the 90s for normal, but its high arrest scores are in the 70s range. This shows that it can predict a normal amount of arrests with my factors very well but struggles a bit for high arrests.

In [ ]:
# Print confusion matrix graph
cm = confusion_matrix(y_test_class, y_pred_class)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Normal", "High-Arrest"], yticklabels=["Normal", "High-Arrest"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix: Normal vs High-Arrest Games")
plt.show()

Im kind of happy with the confusion matrix results with the good results for normal, but has some issues with high arrests games. It seems to be predicting most games to be normal (20 false negatives). Next im going to try and work on improving these results a bit to get better results for high arrests games.

## 4 - Final improvements and further questions